In [1]:
import numpy as np 
import pandas as pd 
import os 
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
amygdala_time_series_df = pd.read_csv("Master_Data_Files/master_df.csv", header= [0,1])
amygdala_time_series_df

subject_0           subject_1           subject_2           subject_3  \
         left     right      left     right      left     right      left   
0   -0.724043 -0.474626 -0.654173 -0.036526 -1.101399  0.255650 -0.665875   
1   -1.241806  0.249143  0.289145 -0.057251 -1.391204  0.726389 -1.046834   
2   -0.586585 -0.587162 -0.721658 -0.894314 -0.440272  1.443644 -1.893001   
3    0.683273  0.236537  0.304811  0.411332  0.772975 -0.096510 -1.336801   
4    0.310507  0.203368  0.230357 -1.433806 -0.026342  0.949549 -1.258972   
..        ...       ...       ...       ...       ...       ...       ...   
523 -0.295987  0.470428  0.838334 -1.121125  1.204350  0.581191 -0.083998   
524 -0.300261  0.827351  0.010546 -1.276964 -0.128276  0.302280  1.221551   
525  1.454496 -0.778130  3.273488 -1.961161 -0.111441 -0.536488 -0.234925   
526  0.341309  0.105951  0.060856 -1.645570 -0.429789  0.074448  0.201250   
527  0.559487  1.058889  0.206309 -1.274101 -0.213287  0.293921  0.850616   

              subject_4            ... subject_25           subject_26  \
        right      left     right  ...       left     right       left   
0    1.384612  0.250878  0.071375  ...  -0.476291 -1.321534  -1.092369   
1    0.760846  1.997693  0.106080  ...  -2.301065  0.889285   0.059911   
2   -1.482707  1.345969  0.196930  ...  -1.896694  0.312375   1.971721   
3   -0.195149  1.341584  0.240217  ...  -0.579941  0.130311   0.336900   
4   -0.671908  0.327529  0.002614  ...   0.196563  0.877104  -0.620652   
..        ...       ...       ...  ...        ...       ...        ...   
523  0.747159  0.786035 -1.200149  ...   1.359247 -0.390796  -0.057738   
524 -0.839555  1.540896  0.569430  ...   0.595575 -0.622517   0.311338   
525  2.725411  1.411557  1.826147  ...   0.224186 -1.128196  -0.049695   
526  0.043955  0.964449  1.067306  ...   1.318863 -2.344816   0.502575   
527 -0.976913  1.600854  0.460463  ...   1.312575 -1.367249   0.367078   

              subject_27           subject_28           subject_29            
        right       left     right       left     right       left     right  
0   -1.344269   1.011459  0.826288  -0.596787  1.481181   0.805425  3.017232  
1    1.328136   1.543867 -0.021904   0.863478  2.288082   1.703162  1.154805  
2   -0.486321  -1.212609  0.075578  -1.176731  1.650930  -0.730842  0.648854  
3   -0.256834   0.996174 -0.192856  -0.792236 -0.703110   1.976824  0.494694  
4   -0.998867  -0.776359  0.474093   0.868483  0.544639  -0.147729  1.464566  
..        ...        ...       ...        ...       ...        ...       ...  
523 -0.707169   0.331933  1.081779  -0.517540 -0.284255   1.321021  1.534357  
524 -0.345230   0.327413  0.884080   0.538849  1.302644  -0.148147 -0.473591  
525 -0.125431  -0.261789  1.448732   0.940224 -0.156144   1.858969  1.876575  
526  0.141820   0.802197  1.349251   0.315911 -0.187054  -0.181112 -0.501251  
527  1.249728   0.759292  0.664094  -0.199954 -0.596935   2.068090  0.721728  

[528 rows x 60 columns]

In [3]:
mean_annotations = pd.read_csv("Master_Data_Files/mean_anxiety_annotations.csv", header=None, names=["mean value"])
mean_annotations

,mean value
0,0.000000
1,-0.612009
2,-0.612009
3,-0.612009
4,-0.612009
...,...
486,-0.742166
487,-0.754869
488,-0.749095
489,-0.683264


In [4]:
max_len = len(mean_annotations)

amygdala_time_series_df = amygdala_time_series_df.iloc[:max_len, :]
amygdala_time_series_df

subject_0           subject_1           subject_2           subject_3  \
         left     right      left     right      left     right      left   
0   -0.724043 -0.474626 -0.654173 -0.036526 -1.101399  0.255650 -0.665875   
1   -1.241806  0.249143  0.289145 -0.057251 -1.391204  0.726389 -1.046834   
2   -0.586585 -0.587162 -0.721658 -0.894314 -0.440272  1.443644 -1.893001   
3    0.683273  0.236537  0.304811  0.411332  0.772975 -0.096510 -1.336801   
4    0.310507  0.203368  0.230357 -1.433806 -0.026342  0.949549 -1.258972   
..        ...       ...       ...       ...       ...       ...       ...   
486 -0.236819 -0.194894 -1.918017  0.011933 -0.646309  0.102246 -0.415507   
487  0.056499  1.297120 -0.775842  0.840575 -0.140510  0.156369  0.855403   
488  0.809064 -0.970546 -1.159404  0.557224 -0.499146  0.845929 -0.708632   
489 -0.780395  0.273168 -1.295235 -0.159257 -0.100304 -0.062793  0.515822   
490 -0.083696 -0.635175  0.115165 -0.466589  1.374471 -0.647842 -0.030748   

              subject_4            ... subject_25           subject_26  \
        right      left     right  ...       left     right       left   
0    1.384612  0.250878  0.071375  ...  -0.476291 -1.321534  -1.092369   
1    0.760846  1.997693  0.106080  ...  -2.301065  0.889285   0.059911   
2   -1.482707  1.345969  0.196930  ...  -1.896694  0.312375   1.971721   
3   -0.195149  1.341584  0.240217  ...  -0.579941  0.130311   0.336900   
4   -0.671908  0.327529  0.002614  ...   0.196563  0.877104  -0.620652   
..        ...       ...       ...  ...        ...       ...        ...   
486 -0.497853  1.413487  2.245959  ...  -0.767336 -1.967725   2.136595   
487 -1.197739  0.130258 -0.362236  ...  -0.413184 -0.530701   0.730927   
488  0.536011 -0.085531 -2.231742  ...   0.035622  1.141761   1.276078   
489  0.243461 -0.615845  1.579207  ...  -1.138821 -0.277449   1.465747   
490  0.173825  0.215597  0.185147  ...   0.486090 -1.026266   2.137252   

              subject_27           subject_28           subject_29            
        right       left     right       left     right       left     right  
0   -1.344269   1.011459  0.826288  -0.596787  1.481181   0.805425  3.017232  
1    1.328136   1.543867 -0.021904   0.863478  2.288082   1.703162  1.154805  
2   -0.486321  -1.212609  0.075578  -1.176731  1.650930  -0.730842  0.648854  
3   -0.256834   0.996174 -0.192856  -0.792236 -0.703110   1.976824  0.494694  
4   -0.998867  -0.776359  0.474093   0.868483  0.544639  -0.147729  1.464566  
..        ...        ...       ...        ...       ...        ...       ...  
486  1.858630  -1.667129 -0.166186   1.922974  0.931832   0.229536 -0.091764  
487  1.130217  -1.835204 -0.189238   1.458164  0.254937   0.642556  0.294516  
488  0.837712  -1.549213 -1.218974   0.141885 -0.582052   1.396469  0.091911  
489  1.341316  -1.670598 -0.743814   1.280481 -0.243784   0.322353 -0.120111  
490  1.676526  -2.043890  0.453334   0.214903 -0.526479   0.927623 -0.241519  

[491 rows x 60 columns]

In [5]:
seperator = amygdala_time_series_df.columns.get_level_values(1)

left_aymgdala_df = amygdala_time_series_df.loc[:, seperator == 'left']
right_aymgdala_df = amygdala_time_series_df.loc[:, seperator == 'right']

In [6]:
X_left = left_aymgdala_df
X_right = right_aymgdala_df

y = mean_annotations

In [7]:
scaler_left = StandardScaler()
X_left_scaled = scaler_left.fit_transform(X_left)

scaler_right = StandardScaler()
X_right_scaled = scaler_right.fit_transform(X_right)

X_left_train, X_left_test, y_left_train, y_left_test = train_test_split(X_left_scaled, y, test_size = 0.2, random_state = 2)
X_right_train, X_right_test, y_right_train, y_right_test = train_test_split(X_right_scaled, y, test_size = 0.2, random_state = 2)

y_left_train = y_left_train.values.ravel()
y_left_test = y_left_test.values.ravel()

y_right_train = y_right_train.values.ravel()
y_right_test = y_right_test.values.ravel()

kernels = ["linear", "poly", "rbf", "sigmoid"]
polys = np.arange(1,10,1)
Cs = [0.1, 1, 10]
gammas = ['scale', 'auto']

results_left = []
results_right = []

for kernel in kernels:
    print(f"The kernel in SVR Model: {kernel}")
    if kernel == "poly":
        for poly in polys:
            for C in Cs:
                for gamma in gammas:
                    print(f"The order of the polynomial in the kernel is: {poly}, C: {C}, gamma: {gamma}")
                    svr_left = SVR(kernel = kernel, degree = poly, C = C, gamma = gamma)
                    svr_left.fit(X_left_train, y_left_train)
                    
                    svr_right = SVR(kernel = kernel, degree = poly, C = C, gamma = gamma)
                    svr_right.fit(X_right_train, y_right_train)
                    
                    y_left_pred_train = svr_left.predict(X_left_train)
                    y_left_pred_test  = svr_left.predict(X_left_test)

                    y_right_pred_train = svr_right.predict(X_right_train)
                    y_right_pred_test  = svr_right.predict(X_right_test)
                    
                    mse_left_train = mean_squared_error(y_left_train, y_left_pred_train)
                    mse_left_test  = mean_squared_error(y_left_test,  y_left_pred_test)
                    
                    mse_right_train = mean_squared_error(y_right_train, y_right_pred_train)
                    mse_right_test  = mean_squared_error(y_right_test,  y_right_pred_test)
                    
                    r2_left_train = r2_score(y_left_train, y_left_pred_train)
                    r2_left_test  = r2_score(y_left_test, y_left_pred_test)
                    
                    r2_right_train = r2_score(y_right_train, y_right_pred_train)
                    r2_right_test  = r2_score(y_right_test, y_right_pred_test)
                    
                    results_left.append({
                        "kernel": kernel,
                        "degree": poly,
                        "C": C,
                        "gamma": gamma,
                        "MSE_train": mse_left_train,
                        "RMSE_train": np.sqrt(mse_left_train),
                        "R2_train": r2_left_train,
                        "MSE_test": mse_left_test,
                        "RMSE_test": np.sqrt(mse_left_test),
                        "R2_test": r2_left_test
                    })
        
                    results_right.append({
                        "kernel": kernel,
                        "degree": poly,
                        "C": C,
                        "gamma": gamma,
                        "MSE_train": mse_right_train,
                        "RMSE_train": np.sqrt(mse_right_train),
                        "R2_train": r2_right_train,
                        "MSE_test": mse_right_test,
                        "RMSE_test": np.sqrt(mse_right_test),
                        "R2_test": r2_right_test
                    })
                    
                    print(f'For Left Amygdala Train Model: MSE = {mse_left_train}, RMSE = {np.sqrt(mse_left_train)}, R2 = {r2_left_train}')
                    print(f'For Left Amygdala Test Model: MSE = {mse_left_test}, RMSE = {np.sqrt(mse_left_test)}, R2 = {r2_left_test}')
                    print("\n")
                    print(f'For Right Amygdala Train Model: MSE = {mse_right_train}, RMSE = {np.sqrt(mse_right_train)}, R2 = {r2_right_train}')
                    print(f'For Right Amygdala Test Model: MSE = {mse_right_test}, RMSE = {np.sqrt(mse_right_test)}, R2 = {r2_right_test}')
                    print("\n")
    else:
        for C in Cs:
                for gamma in gammas:
                    print(f"C: {C}, gamma: {gamma}")
                    svr_left = SVR(kernel = kernel, C = C, gamma = gamma)
                    svr_left.fit(X_left_train, y_left_train)
                    
                    svr_right = SVR(kernel = kernel, C = C, gamma = gamma)
                    svr_right.fit(X_right_train, y_right_train)
                    
                    y_left_pred_train = svr_left.predict(X_left_train)
                    y_left_pred_test  = svr_left.predict(X_left_test)

                    y_right_pred_train = svr_right.predict(X_right_train)
                    y_right_pred_test  = svr_right.predict(X_right_test)
                    
                    mse_left_train = mean_squared_error(y_left_train, y_left_pred_train)
                    mse_left_test  = mean_squared_error(y_left_test,  y_left_pred_test)
                    
                    mse_right_train = mean_squared_error(y_right_train, y_right_pred_train)
                    mse_right_test  = mean_squared_error(y_right_test,  y_right_pred_test)
                    
                    r2_left_train = r2_score(y_left_train, y_left_pred_train)
                    r2_left_test  = r2_score(y_left_test, y_left_pred_test)
                    
                    r2_right_train = r2_score(y_right_train, y_right_pred_train)
                    r2_right_test  = r2_score(y_right_test, y_right_pred_test)
                    
                    results_left.append({
                        "kernel": kernel,
                        "degree": None,
                        "C": C,
                        "gamma": gamma,
                        "MSE_train": mse_left_train,
                        "RMSE_train": np.sqrt(mse_left_train),
                        "R2_train": r2_left_train,
                        "MSE_test": mse_left_test,
                        "RMSE_test": np.sqrt(mse_left_test),
                        "R2_test": r2_left_test
                    })
        
                    results_right.append({
                        "kernel": kernel,
                        "degree": None,
                        "C": C,
                        "gamma": gamma,
                        "MSE_train": mse_right_train,
                        "RMSE_train": np.sqrt(mse_right_train),
                        "R2_train": r2_right_train,
                        "MSE_test": mse_right_test,
                        "RMSE_test": np.sqrt(mse_right_test),
                        "R2_test": r2_right_test
                    })
                    
                    print(f'For Left Amygdala Train Model: MSE = {mse_left_train}, RMSE = {np.sqrt(mse_left_train)}, R2 = {r2_left_train}')
                    print(f'For Left Amygdala Test Model: MSE = {mse_left_test}, RMSE = {np.sqrt(mse_left_test)}, R2 = {r2_left_test}')
                    print("\n")
                    print(f'For Right Amygdala Train Model: MSE = {mse_right_train}, RMSE = {np.sqrt(mse_right_train)}, R2 = {r2_right_train}')
                    print(f'For Right Amygdala Test Model: MSE = {mse_right_test}, RMSE = {np.sqrt(mse_right_test)}, R2 = {r2_right_test}')
                    print("\n")

The kernel in SVR Model: linear
C: 0.1, gamma: scale
For Left Amygdala Train Model: MSE = 0.3774765780533287, RMSE = 0.6143912255666814, R2 = 0.09576200967989934
For Left Amygdala Test Model: MSE = 0.4557398490968081, RMSE = 0.6750850680446192, R2 = -0.04001382018534683


For Right Amygdala Train Model: MSE = 0.3720371736380641, RMSE = 0.6099485008081126, R2 = 0.10879199989111266
For Right Amygdala Test Model: MSE = 0.4380849385558196, RMSE = 0.6618798520546003, R2 = 0.00027528552518685245


C: 0.1, gamma: auto
For Left Amygdala Train Model: MSE = 0.3774765780533287, RMSE = 0.6143912255666814, R2 = 0.09576200967989934
For Left Amygdala Test Model: MSE = 0.4557398490968081, RMSE = 0.6750850680446192, R2 = -0.04001382018534683


For Right Amygdala Train Model: MSE = 0.3720371736380641, RMSE = 0.6099485008081126, R2 = 0.10879199989111266
For Right Amygdala Test Model: MSE = 0.4380849385558196, RMSE = 0.6618798520546003, R2 = 0.00027528552518685245


C: 1, gamma: scale
For Left Amygdala Tr

In [8]:
results_left_amygdala_df = pd.DataFrame(results_left)
print(results_left_amygdala_df)

output_folder = "Supplementary_Materials"
results_left_amygdala_df.to_csv(f"{output_folder}/left_amygdala_SVR_model_parameters.csv", index=False)

     kernel  degree     C  gamma  MSE_train  RMSE_train   R2_train  MSE_test  \
0    linear     NaN   0.1  scale   0.377477    0.614391   0.095762  0.455740   
1    linear     NaN   0.1   auto   0.377477    0.614391   0.095762  0.455740   
2    linear     NaN   1.0  scale   0.376774    0.613819   0.097445  0.455393   
3    linear     NaN   1.0   auto   0.376774    0.613819   0.097445  0.455393   
4    linear     NaN  10.0  scale   0.376816    0.613853   0.097345  0.455280   
..      ...     ...   ...    ...        ...         ...        ...       ...   
67  sigmoid     NaN   0.1   auto   0.408922    0.639470   0.020436  0.457354   
68  sigmoid     NaN   1.0  scale   0.868775    0.932081  -1.081134  0.609329   
69  sigmoid     NaN   1.0   auto   0.842152    0.917689  -1.017360  0.585462   
70  sigmoid     NaN  10.0  scale  27.223137    5.217580 -64.212509  5.354092   
71  sigmoid     NaN  10.0   auto  27.022294    5.198297 -63.731393  5.392243   

    RMSE_test    R2_test  
0    0.67508

In [9]:
results_right_amygdala_df = pd.DataFrame(results_right)
print(results_right_amygdala_df)

output_folder = "Supplementary_Materials"
results_right_amygdala_df.to_csv(f"{output_folder}/right_amygdala_SVR_model_parameters.csv", index=False)

     kernel  degree     C  gamma  MSE_train  RMSE_train    R2_train  MSE_test  \
0    linear     NaN   0.1  scale   0.372037    0.609949    0.108792  0.438085   
1    linear     NaN   0.1   auto   0.372037    0.609949    0.108792  0.438085   
2    linear     NaN   1.0  scale   0.372182    0.610067    0.108445  0.435113   
3    linear     NaN   1.0   auto   0.372182    0.610067    0.108445  0.435113   
4    linear     NaN  10.0  scale   0.372261    0.610132    0.108255  0.434935   
..      ...     ...   ...    ...        ...         ...         ...       ...   
67  sigmoid     NaN   0.1   auto   0.401452    0.633603    0.038329  0.424124   
68  sigmoid     NaN   1.0  scale   0.862573    0.928748   -1.066277  0.520451   
69  sigmoid     NaN   1.0   auto   0.885993    0.941272   -1.122379  0.526759   
70  sigmoid     NaN  10.0  scale  40.241642    6.343630  -95.398090  2.583737   
71  sigmoid     NaN  10.0   auto  42.965384    6.554799 -101.922763  3.626273   

    RMSE_test   R2_test  
0

In [10]:
min_rmse_left = results_left_amygdala_df["RMSE_test"].min()
best_model_left_rmse = results_left_amygdala_df[results_left_amygdala_df["RMSE_test"] == min_rmse_left]
print("Left Amygdala - Best RMSE Test Model:\n", best_model_left_rmse)

max_r2_left = results_left_amygdala_df["R2_test"].max()
best_model_left_r2 = results_left_amygdala_df[results_left_amygdala_df["R2_test"] == max_r2_left]
print("\nLeft Amygdala - Best R2 Test Model:\n", best_model_left_r2)

Left Amygdala - Best RMSE Test Model:
    kernel  degree    C gamma  MSE_train  RMSE_train  R2_train  MSE_test  \
63    rbf     NaN  1.0  auto   0.097036    0.311506  0.767553   0.37419   

    RMSE_test   R2_test  
63    0.61171  0.146087  

Left Amygdala - Best R2 Test Model:
    kernel  degree    C gamma  MSE_train  RMSE_train  R2_train  MSE_test  \
63    rbf     NaN  1.0  auto   0.097036    0.311506  0.767553   0.37419   

    RMSE_test   R2_test  
63    0.61171  0.146087  


In [11]:
min_rmse_right = results_right_amygdala_df["RMSE_test"].min()
best_model_right_rmse = results_right_amygdala_df[results_right_amygdala_df["RMSE_test"] == min_rmse_right]
print("\nRight Amygdala - Best RMSE Test Model:\n", best_model_right_rmse)

max_r2_right = results_right_amygdala_df["R2_test"].max()
best_model_right_r2 = results_right_amygdala_df[results_right_amygdala_df["R2_test"] == max_r2_right]
print("\nRight Amygdala - Best R2 Test Model:\n", best_model_right_r2)


Right Amygdala - Best RMSE Test Model:
    kernel  degree    C  gamma  MSE_train  RMSE_train  R2_train  MSE_test  \
62    rbf     NaN  1.0  scale    0.09401     0.30661  0.774802  0.413402   

    RMSE_test   R2_test  
62   0.642963  0.056603  

Right Amygdala - Best R2 Test Model:
    kernel  degree    C  gamma  MSE_train  RMSE_train  R2_train  MSE_test  \
62    rbf     NaN  1.0  scale    0.09401     0.30661  0.774802  0.413402   

    RMSE_test   R2_test  
62   0.642963  0.056603  
